# 2. Feature Extraction and Description

>1. **Reduce** the amount of data
>2. **Preserve** useful information (e.g. edge, corner, shape)
>3. **Discard** redundant information (e.g. lighting conditions)

## 2.1. 1D Edge Detection
* **Step 1. Smooth the signal to suppress noise**

  * Convolve the signal $I(x)$ with a Gaussian kernel $g_\sigma(x)$
  * **Small $\sigma \rightarrow$** Large cutoff frequency $\rightarrow$ **Preserve detail**
  * **Large $\sigma \rightarrow$** Small cutoff frequency $\rightarrow$ **Suppress detail**

>$$g_\sigma(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{x^2}{2\sigma^2}\right)$$
>$$\;$$
>\begin{align}
s(x)=g_\sigma(x) * I(x)&=\int_{-\infty}^{\infty}g_\sigma(u)I(x-u)du\\
&=\int_{-\infty}^{\infty}g_\sigma(x-u)I(u)du
\end{align}

* **Step 2-a. Compute $s'(x)$ and look for maxima & minima**

  * From the derivative theorem of convolution,
  
>$$s'(x)=\frac{d}{dx} [ g_\sigma(x) * I(x) ] = g'_\sigma(x)*I(x)$$

* **Step 2-b. Compute $s''(x)$ and look for zero-crossings**

  * The signal is convolved with the **Laplacian of a Gaussian**

>$$s''(x)=g''_\sigma(x)*I(x)$$



## 2.2. 2D Edge Detection - Canny (1986)
* **Step 1. Convolve with a 2D Gaussian $G_\sigma(x,y)$**

>\begin{align}
G_\sigma(x,y)&=\frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)\\
\\
S(x,y)&=G_\sigma(x,y)*I(x,y)\\
\\
&=\int_{-\infty}^{\infty}\int_{-\infty}^{\infty}G_\sigma(u,v)I(x-u,y-v)du dv
\end{align}

* **Step 2. Find the Gradient of $S(x,y)$**

>\begin{align}
\nabla S &=\nabla (G_\sigma*I) \\
&=
\begin{bmatrix}
    \frac{\partial (G_\sigma *I)}{\partial x} \\
    \frac{\partial (G_\sigma *I)}{\partial y} \\
\end{bmatrix} \\
&=
\begin{bmatrix}
    \frac{\partial G_\sigma}{\partial x} *I \\
    \frac{\partial G_\sigma}{\partial y} *I \\
\end{bmatrix}
\end{align}

* **Step 3. Non-maximal Suppression**

>$$\text{Edge elements (edgels) are placed where } | \nabla S | \text{ is greater than local values in the direction of } \pm \nabla S $$
  
* **Step 4. Threshold the Edgels**

>$$\text{Output: edgel positions, each with strength } |\nabla S| \text{ and orientation } \nabla S /|\nabla S|$$

## 2.3. 2D Edge Detection - Marr and Hildreth (1980)
* Unlike Canny edge detector, Marr-Hildreth operator is **isotropic**
* **Algorithm: Find the zero-crossings of:**

>$$\nabla^2G_\sigma*I$$
>$$\;$$
>$$\text{where } \nabla^2G_\sigma=\left( \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} \right) G_\sigma \text{ is the Laplacian of } G_\sigma$$

## 2.4. Corner Detection - Harris (1987)

* **Corner Detection** - Important in **tracking** objects and **matching** stereo pairs
* Important in understanding **kinetic effect** and **ego-motion**
* **Step 1. Calculate the change in intensity in direction $\textbf{n}$**

  * Here, $I$ is already smoothed

>\begin{align}
I_n &\equiv \nabla I(x,y) \cdot \widehat{\textbf{n}} \equiv \begin{bmatrix} I_x & I_y \end{bmatrix}^T \cdot \widehat{\textbf{n}}\\
\;\\
I_n^2 &= \frac{\textbf{n}^T \nabla I \nabla I^T \textbf{n}}{\textbf{n}^T\textbf{n}}\\
\;\\
&= \frac{\textbf{n}^T \begin{bmatrix} 
I_x^2 & I_xI_y \\
I_xI_y & I_y^2
\end{bmatrix}
\textbf{n}}{\textbf{n}^T\textbf{n}}
\end{align}
>$$\;$$
>$$\text{where } I_x \equiv \frac{\partial I}{\partial x} \text{ and } I_y \equiv \frac{\partial I}{\partial y}$$

* **Step 2. Smooth $I_n^2$ by convolution with a Gaussian kernel**

>\begin{align}
C_n(x,y) &= G_\sigma (x,y)*I^2_n \\
\;\\
&= \frac{\textbf{n}^T \begin{bmatrix} 
\left\langle I_x^2 \right\rangle & \left\langle I_xI_y \right\rangle\\
\left\langle I_xI_y \right\rangle & \left\langle I_y^2 \right\rangle\\
\end{bmatrix}
\textbf{n}}{\textbf{n}^T\textbf{n}}
\end{align}
>$$\;$$
>$$\text{where } \left\langle \right\rangle \text{ is the smoothed value}$$

* **Step 3. Use the eigenvalues of $\text{A}$ to determine the structure (Rayleigh's Quotient)**

>$$C_n(x,y)=\frac{\textbf{n}^T \text{A} \textbf{n}}{\textbf{n}^T\textbf{n}}$$
>$$\;$$
>$$\text{where,} \;\;\; \text{A}=\begin{bmatrix} 
\left\langle I_x^2 \right\rangle & \left\langle I_xI_y \right\rangle\\
\left\langle I_xI_y \right\rangle & \left\langle I_y^2 \right\rangle\\
\end{bmatrix}$$
>$$\;$$
>$$\text{then,} \;\;\; \lambda_1 \leq C_n(x,y) \leq\lambda_2$$
>
>* **$\text{A}$**: Structure Tensor
>* **No structure**(smooth): $\lambda_1 \approx \lambda_2 \approx 0$
>* **1D structure**(edge): $\lambda_1 \approx 0$ (direction of edge), $\lambda_2$ is large (normal to edge)
>* **2D structure**(corner): $\lambda_1$ and $\lambda_2$ both large and distinct

* **Step 4. If $M_c$ exceeds some threshold $\rightarrow$ Mark corners**

>$$M_c=\lambda_1 \lambda_2-\kappa(\lambda_1+\lambda_2)^2$$
>$$\;$$
>$$M_c=\det{\text{A}} - \kappa \text{ tr A}$$
>* The second form is preferable (low computational cost)
>* $\kappa$: generally range from $0.04$ to $0.15$

## 2.5. Blob Detection

* **Blob:** Area of uniform intensity in the image

>* Blobs are usually found inside of objects (as opposed to edges)
>* Thus they are less likely to contain background in queries
>* +) stability, repeatability, definite optimal scale, ...

* **Method 1: Laplacian of Gaussian**

>* Convolve with the **Laplacian of Gaussian** $\rightarrow$ Locate minimum
>* As $\sigma$ increases, larger image features are detected

* **Method 2: Difference of Gaussians**

>* Use DoG response to approximate the Laplacian of Gaussian (high $\sigma$ - small $\sigma$)
>* **Scale space pyramid** $\rightarrow$ subtract one member from the one directly above it

## 2.6. Scale Space
* Achieve scale independence by looking at different resolutions
* **Scale Space**

>\begin{align}
L(x,y,t)&=G(x,y,t)*I(x,y)\\
\;\\
G(x,y,t)&=\frac{1}{2\pi t}\exp{-\frac{x^2+y^2}{2t}}\\
\;\\
t&=\sigma^2
\end{align}

* **Choose discrete set of low-pass filter**

>$$\sigma_i=2^\frac{1}{s}\sigma_{i-1}=2^\frac{i}{s}\sigma_0$$
>
>* $\sigma$ doubles after $s$ intervals ($=$ an **octave**)
>* Avoid blurring with large scales by **subsampling** the image after each octave
>* $\Rightarrow$ **Image Pyramid**

* **Within each octave, convolve repeatedly**

>$$G(\sigma_1)*G(\sigma_2)=G\left(\sqrt{\sigma^2_1+\sigma^2_2}\right)$$
>
>* The following should be satisfied:
>
>$$G(\sigma_{i+1})=G(\sigma_i)*G(\sigma_k)$$
>
>* $\sigma_k$ can be calculated
>
>\begin{align}
\sigma_k&=\sqrt{\sigma_{i+1}^2-\sigma_i^2}\\
\sigma_{i+1}&=2^{\frac{1}{s}}\sigma_i\\
\sigma_k&=\sigma_i\sqrt{2^{\frac{2}{s}}-1}
\end{align}

* **Ideal scale for a keypoint is located at the maximum of the scale space**
  * $\Rightarrow$ The largest value of the samples in the pyramid is obtained and interpolated

## 2.7. Matching Image Patches

* **Normalised Cross-Correlation** between an image patch $P(x,y)$ and other image portion $I(x,y)$

>$$c(x,y)=\frac{\sum^n_{u=-n}\sum^n_{v=-n}P(u,v)I(x+u,y+v)}{\sqrt{\sum^n_{u=-n}\sum^n_{v=-n}P^2(u,v)\sum^n_{u=-n}\sum^n_{v=-n}I^2(x+u,y+v)}}$$

* **Matching intensity patches**

>$$CC(P_1, P_2)=\sum_i^N{P_1[i]P_2[i]}$$
>
>* Not robust to changes (brightness & contrast)

* **Zero Normalised Patches**

>$$ZN(x,y)=\frac{Z(x,y)}{\sigma}=\frac{I(x,y)-\mu}{\sigma}$$
>
>* Robust to changes
>* Patches can be matched using simple cross-correlation

## 2.8. SIFT(Scale Invariant Feature Transform)

* **SIFT Keypoint Detection**

>* **Step 1:** Obtain DoG images across different scales
>  * If the image pyramid is available $\rightarrow$ subtract an image from the one above it
>* **Step 2:** Compare each pixel to its neighbours (in the same and neighbouring scales) to find local extrema
>* **Step 3:** Discard the candidates with low-contrast (sensitive to noise)
>* **Step 4:** Discard the candidates on the edges (sensitive to background image)
>* **Step 5:** Calculate the orientations of the candidates from the direction of the intensity gradient

* **SIFT Interest Point Descriptor**

>* **Step 1:** Extract a $N \times N$ patch at the given keypoint (typically $N=16$)
>  * Area of patch: selected in accordance with the scale of the keypoint
>* **Step 2:** Calculate the intensity gradient in each pixel
>* **Step 3:** Divide the patch into $c$ cells (typically $c=16$)
>* **Step 4:** Within each cell, bin the intensity gradients into $d$ directions
>  * In this process, the gradients are weighted by the Gaussian window of $\sigma$
>  * $\Rightarrow$ robustness to occlusion / $\sigma$: half the scale of the interest point
>* **Step 5:** Normalise the resulting $(d \times c)$-dim vector
>  * $\Rightarrow$ robustness to affine lighting changes (e.g. brightness and contrast)
>* **Step 6:** Apply threshold of $0.2$ and re-normalise
>  * $\Rightarrow$ robustness to non-affine lighting changes

## 2.9. Texture
* **What is Texture?**

>* **Image Texture** consists of organised patterns of regular sub-elements called **textons**

* **Characterising Texture**

>* **Filter bank**:
>  * $8$ Lablacian of Gaussian filters
>  * $4$ Gaussian filters at different scales
>  * $36$ Oriented filters
>    * $6$ Different angles
>    * $3$ Different scales
>    * $2$ Different phases ($1^{st} \text{&}\; 2^{nd}$ derivatives of Gaussians on the minor axis)
>* **Descriptor**:
>  * Simply the concatenated responses of all the filters at a pixel
>  * This is innately immune to most changes in an image

## 2.10. Implementation Details
* **Convolution as Truncated Summations**

>$$S(x,y)=\sum_{u=-n}^{n} \sum_{v=-n}^{n} G_\sigma(u,v)I(x-u,y-v)$$
>
>* Kernels are trucated so that the discarded samples are less than $1/1000$ of the peak value
>* $\pm 3\sigma$ is often enough / smaller $n$ leads to sharp discontinuity

* **2D Convolution $\rightarrow$ Two 1D Convolutions**

>$$G_\sigma(x,y)*I(x,y)=g_\sigma(x)*[g_\sigma(y)*I(x,y)]$$
>
>$$\mathcal{O} \left((2n+1)^2 \right) \text{ vs. } \mathcal{O} \left(2(2n+1)\right)$$

* **Differentiation** (approximation using **Taylor Expansion**)

>$$\frac{\partial S}{\partial x} \approx \frac{S(x+1,y)-S(x-1,y)}{2}$$
>
>* Equivalent to convolving the rows of image samples with the kernel $(1/2, 0, -1/2)$